# PyTorch를 이용한 경사 하강법과 선형 회귀

### "Deep Learning with Pytorch: Zero to GANs"의 2번째 파트

>이 튜토리얼은 [PyTorch](https://pytorch.org)를 이용한 초보자용 딥러닝 학습 튜토리얼 입니다.   
>학습하기 위한 최고의 방법은 본인이 코드를 실행하고, 실험해 보는 것이기 때문에 이 튜토리얼은 실용성과 코딩 중심으로 진행됩니다.

이번 튜토리얼에선 다음과 같은 주제를 다룹니다:

- __선형 회귀와 경사 하강법__의 소개
- 선형 회귀 모델을 PyTorch의 텐서를 사용한 구현
- __경사 하강법을 이용한 선형 회귀 모델 학습__
- PyTorch 기본 함수를 이용한 경사 하강법과 선형 회귀 구현

시작하기 전, 필요한 라이브러리들을 설치해야만 합니다.<br>
PyTorch의 설치는 여러분의 실행 시스템/클라우드 환경에 따라 다릅니다.<br>
자세한 설치 명령어의 내용은 다음 주소에서 확인 할 수 있습니다: https://pytorch.org

In [56]:
# 필요한 경우 주석 처리를 제거하고 여러분의 운영 체제에 적합한 명령을 실행하시면 됩니다.

# Linux / Binder
# !pip install numpy torch==1.7.0+cpu torchvision==0.8.1+cpu torchaudio==0.7.0 -f https://download.pytorch.org/whl/torch_stable.html

# Windows
# !pip install numpy torch==1.7.0+cpu torchvision==0.8.1+cpu torchaudio==0.7.0 -f https://download.pytorch.org/whl/torch_stable.html

# MacOS
# !pip install numpy torch torchvision torchaudio

---
## 1. 선형 회귀

>이번 튜토리얼에서 저희는 _선형 회귀_ 라는 머신러닝 알고리즘의 기초에 대해 알아볼 것입니다.<br>
>저희는 각 지역의 평균 기온, 강수량, 습도 (_입력 변수_)를 보고 사과와 오렌지의 수확량(_목표 변수_)을 예측하는 모델을 만들 것입니다.<br>
아래 그림은 학습 데이터의 일부입니다.

![선형 회귀 학습 데이터](https://i.imgur.com/6Ujttb4.png)

선형 회귀 모델에서, 각 목표 변수는 입력 변수의 <font color=red>가중 합계(weighted sum)</font>로 추정되고 편향(bias)이라는 변수로 상쇄됩니다 : 

```
yield_apple  = w11 * temp + w12 * rainfall + w13 * humidity + b1
yield_orange = w21 * temp + w22 * rainfall + w23 * humidity + b2
```

아래에 있는 전체 학습 데이터로 학습한 선형 회귀 그래프를 보면 사과의 수확량이 온도, 강우 및 습도라는 변수의 선형 또는 평면 함수임을 확인 할 수 있습니다:

![선형 회귀 그래프](https://i.imgur.com/4DJ9f8X.png)

선형 회귀의 _학습_ 은 학습 데이터를 사용하여 가중치 집합 `w11, w12, ... w23, b1 & b2`를 파악하여 새 데이터를 정확하게 예측하는 것입니다.<br>
_학습_ 가중치는 새로운 지역의 <font color=red>평균 온도, 강수량, 습도</font>를 사용해 그 지역의 수확량을 예측하는데 사용됩니다.

저희는 _경사 하강법_ 이라는 최적화 기법을 통해 더 나은 예측을 만들기 위해 모델의 가중치를 여러번 조정해 모델을 훈련시킬 것입니다. <br>
먼저 __Numpy__ 와 __PyTorch__ 를 import하는 것부터 시작합니다.

In [1]:
import numpy as np
import torch

---
## 2. 훈련 데이터

>저희는 `inputs`과 `targets`라는 2개의 행렬을 사용해 훈련 데이터를 표현할 수 있습니다.<br>
>각 지역당 행이 1개씩 있고, 변수당 열이 1개씩 있습니다.

In [2]:
# Input (평균 온도, 강수량, 습도)
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70]], dtype='float32')

In [3]:
# Targets (사과, 오렌지)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119]], dtype='float32')

input과 target을 따로 사용할것이기에 분리 했습니다. 또한 저희는 일반적인 학습 데이터로 작업하는 방식인 numpy 배열로 만들었습니다:<br>
일부 CSV파일을 numpy 배열로 읽고 전처리를 한 후 PyTorch 텐서로 변환 합니다.

__이제 PyTorch 텐서로 배열을 변환해 봅시다!__

In [4]:
# inputs와 targets를 텐서로 변환하기
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)
print(inputs)
print(targets)

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.]])
tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


---
## 3. 선형회귀 모델 밑바닥부터 구현해보기

>가중치와 편향인 (`w11, w12,... w23, b1 & b2`)는 임의의 값으로 초기화된 행렬로 표현 가능합니다. <br>
`w`의 첫 행과 `b`의 첫 요소는 첫 타겟 변수를 예측하기 위해 필요하다. <br> (즉, 사과의 수확량을 예측하는데 사용됨. 두 번째는 오렌지의 수확량을 예측하는데 사용)

In [5]:
# 가중치와 편향
w = torch.randn(2, 3, requires_grad=True)
b = torch.randn(2, requires_grad=True)
print(w)
print(b)

tensor([[-0.6300,  1.6517, -1.0690],
        [-0.8544, -0.3654,  0.9815]], requires_grad=True)
tensor([1.4770, 0.7812], requires_grad=True)


`torch.randn`은 주어진 형태(shape)에 맞추어 평균 0, 표준편차 1인 [정규 분포](https://en.wikipedia.org/wiki/Normal_distribution)에서 임의로 값을 가져와 텐서를 생성합니다. 

저희 *모델*은 단순히 `inputs` 행렬과 가중치 `w`(전치)의 행렬 곱 후 편향인 `b`를 더하는 작업을 수행하게 됩니다. 

![matrix-mult](https://i.imgur.com/WGXLFvA.png)

저희는 모델을 다음과 같이 정의 할 수 있습니다:

In [16]:
def model(x):
    return x @ w.t() + b

>`tensor @ tensor`는 Pytorch의 행렬 곱으로 표현되고, <br>
`tensor.t()`는 텐서의 전치를 리턴하는 메서드입니다.

입력 데이터를 모델에 전달하여 얻은 행렬은 목표 변수(사과와 오렌지의 수확량)에 대한 예측입니다.

In [17]:
# 예측 생성
preds = model(inputs)
print(preds)

tensor([[ 20.1856, -43.8673],
        [ 21.0823, -46.3079],
        [105.9955, -65.5871],
        [-31.3104, -65.7659],
        [ 41.7411, -24.5442]], grad_fn=<AddBackward0>)


이제 저희 모델과 실제 값의 차이를 비교해 봅시다.

In [18]:
# targets와 비교
print(targets)

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


저희 모델은 무작위로 가중치와 편향을 초기화했기 때문에 모델의 예측과 실제 값 사이에 큰 차이가 있음을 확인할 수 있습니다.

---
## 4. 손실 함수

>모델을 개선하기 전에, 저희는 모델이 얼마나 잘 작동하는지에 대해 평가할 수 있는 방법이 필요합니다.<br>
저희는 다음과 같은 방법들로 저희 모델의 예측과 실제 목표값을 비교할 수 있습니다:

* `예측`과 `타겟` 행렬의 차이를 계산
* `예측`과 `타겟` 행렬의 차이 행렬의 모든 요소를 제곱해 음수 제거
* 결과 행렬의 요소의 평균을 계산

결과는 **mean squared error** (MSE)로 계산됩니다.

In [19]:
# MSE loss
def mse(t1, t2):
    diff = t1 - t2
    return torch.sum(diff * diff) / diff.numel()

>`torch.sum(tensor)`은 텐서의 모든 원소의 합을 반환합니다.<br>
`tensor.numel` 메서드는 텐서의 원소 수를 반환합니다.<br>
- 이제 현재 우리 모델의 예측의 mean squared error를 계산해 봅시다.

In [20]:
# loss 계산하기
loss = mse(preds, targets)
print(loss)

tensor(11690.4551, grad_fn=<DivBackward0>)


__결과를 해석하는 방법:__<br>
>_평균적으로 예측의 각 요소는 손실의 제곱근에 의해 실제 목표와 다릅니다_.<br>
그리고 우리가 예측하려는 숫자가 50 ~ 200 범위에 있다는 점을 생각해보면 매우 안좋습니다.<br>
결과는 모델이 목표 변수를 예측하는데 얼마나 나쁜지를 나타내기 때문에 _loss_ 라고 합니다.<br>
__loss 는 모델의 정보 손실을 나타내고 이 손실이 적을수록 모델은 더 좋습니다.__

---
## 5. 경사(gradient) 계산하기

Pytorch를 사용하면 가중치와 편향의 `requires_grad`를 `True`로 설정하여 자동으로 가중치와 편향의 기울기(gradient)를 계산할 수 있습니다.<br>
저희는 잠시후 이게 어떻게 유용한지 알아보겠습니다.

In [21]:
# 경사 계산하기
loss.backward()

각 텐서의 경사(gradient)는 `.grad`라는 변수에 저장되어 있습니다. (가중치 행렬의 도함수는 동일한 차원의 행렬이라는 점에 유의)

In [22]:
# 가중치의 경사
print(w)
print(w.grad)

tensor([[-0.6300,  1.6517, -1.0690],
        [-0.8544, -0.3654,  0.9815]], requires_grad=True)
tensor([[ -7545.1509,  -7035.2490,  -4955.8481],
        [-23752.4316, -26160.8184, -15876.9922]])


---
## 6. 손실을 줄이도록 가중치와 편향을 조정하기

손실은 저희의 가중치와 편향의 [2차 함수](https://en.wikipedia.org/wiki/Quadratic_function)입니다. <br>
저희의 목표는 손실이 가장 낮은 가중치 집합을 찾는 것입니다.
개별 가중치, 편향 요소의 손실 그래프를 그린다면 아래 그림과 같이 보일 것입니다.<br>
중요한 것은 경사가 손실의 [변화율](https://en.wikipedia.org/wiki/Slope)을 나타낸다는 것입니다.

만약 경사 요소가 __양__ 일 경우:

* 가중치 요소의 값의 __증가__ 는 손실의 __증가__ 를 일으킵니다.
* 가중치 요소의 값의 __감소__ 는 손실의 __감소__ 를 일으킵니다.


![양의 경사도](https://i.imgur.com/WLzJ4xP.png)

만약 경사 요소가 __음__ 일 경우:

* 가중치 요소의 값의 __증가__ 는 손실의 __감소__ 를 일으킵니다.
* 가중치 요소의 값의 __감소__ 는 손실의 __증가__ 를 일으킵니다.

![음의 경사도](https://i.imgur.com/dvG2fxU.png)

가중치 요소를 변경함으로 인한 손실의 증가 또는 감소는 해당 요소에 대한 손실의 경사와 비례합니다. <br>
이러한 현상은 모델을 향상시키기 위해 저희가 사용하는 _경사 하강_ 최적화 알고리즘에 기반해 형성됩니다.<br>

손실을 약간 줄이기 위해 각 가중치 요소에서 손실 기울기에 비례하는 작은 값을 뺄 수 있습니다.

In [25]:
w
w.grad

tensor([[ -7545.1509,  -7035.2490,  -4955.8481],
        [-23752.4316, -26160.8184, -15876.9922]])

In [26]:
with torch.no_grad():
    w -= w.grad * 1e-5
    b -= b.grad * 1e-5

가중치를 많이 수정하지 않기 위해 경사를 매우 작은 수(본 튜토리얼에서는 `10^-5`)를 곱해 빼줍니다.<br>
경사로의 내리막 방향으로 아주 작게 움직이고 싶기 때문입니다. 이 작은 수를 _learning rate_ 라 부릅니다.

가중치와 편향을 수정하는 동안 `torch.no_grad`를 사용해 경사를 추적, 계산, 수정하지 않아야 한다는것을 PyTorch에게 알려줄 수 있습니다.

In [27]:
# 손실의 변화를 확인해보기
loss = mse(preds, targets)
print(loss)

tensor(11690.4551, grad_fn=<DivBackward0>)


계속하기 전, `.zero_()` 메서드를 호출해 경사를 0으로 재설정합니다. <br>
PyTorch가 경사를 누적하기 때문에 이 `.zero()`의 호출은 꼭 수행되어야합니다.<br>
그렇지 않으면 다음 번 손실에 대해 `.backward`를 호출시 새로운 경사 값이 기존 경사 값에 추가되어 잘못된 결과가 나올 수 있습니다.

In [28]:
w.grad.zero_()
b.grad.zero_()
print(w.grad)
print(b.grad)

tensor([[0., 0., 0.],
        [0., 0., 0.]])
tensor([0., 0.])


---
## 7. 경사 하강법을 이용한 모델 학습

위에서 봤듯이, 경사 하강법 알고리즘으로 손실(loss)을 줄이고 모델을 향상시켰습니다.<br>
따라서, 저희는 다음과 같은 절차로 __학습__ 을 진행 할 수 있습니다:

>1. 예측 생성
>2. 손실 계산
>3. 가중치와 편향의 경사 계산
>4. 계산된 경사에 작은 값을 곱한 값을 가중치에서 빼서 가중치 조정
>5. 가중치를 0으로 초기화

이제 위의 절차를 구현해봅시다.

In [30]:
# 1.예측 생성
preds = model(inputs)
print(preds)

tensor([[ 32.5395,  -2.1689],
        [ 37.3125,   8.4938],
        [124.8628,  -0.6541],
        [-18.7542, -24.4106],
        [ 57.1715,  28.0775]], grad_fn=<AddBackward0>)


In [31]:
# 2. 손실 계산
loss = mse(preds, targets)
print(loss)

tensor(4992.1826, grad_fn=<DivBackward0>)


In [32]:
# 3. 경사 계산
loss.backward()
print(w.grad)
print(b.grad)

tensor([[-2499.4419, -2156.5405, -1636.1316],
        [-7570.3652, -8450.9434, -5082.4717]])
tensor([-29.5736, -90.1324])


위에서 계산한 경사를 이용해 가중치와 편향을 업데이트 합니다.

In [33]:
# 4.가중치를 업데이트 & 경사 초기화
with torch.no_grad():
    w -= w.grad * 1e-5
    b -= b.grad * 1e-5
    w.grad.zero_()
    b.grad.zero_()

새로운 가중치와 편향을 확인해 봅시다.

In [34]:
print(w)
print(b)

tensor([[-0.5295,  1.7436, -1.0031],
        [-0.5412, -0.0193,  1.1911]], requires_grad=True)
tensor([1.4786, 0.7863], requires_grad=True)


새로운 가중치와 편향으로 모델은 더 작은 손실(loss)을 가질 것입니다.

In [36]:
# Calculate loss
preds = model(inputs)
loss = mse(preds, targets)
print(loss)

tensor(3461.1367, grad_fn=<DivBackward0>)


경사 하강법을 사용해 가중치와 편향을 약간 조절하여 많은 손실을 줄였습니다.

---
## 8. 여러 epoch동안 학습하기

손실을 더 줄이기 위해, 위에서 진행한 작업을 수차례 반복할 수 있습니다.<br>
이때 각 반복을 _epoch_ 라 부릅니다. 이제 100 epochs 동안 모델을 학습해 봅시다.

In [37]:
# 100 epochs 동안 학습
for i in range(100):
    preds = model(inputs)
    loss = mse(preds, targets)
    loss.backward()
    with torch.no_grad():
        w -= w.grad * 1e-5
        b -= b.grad * 1e-5
        w.grad.zero_()
        b.grad.zero_()

다시 한번, 손실이 작아졌는지 확인해 봅시다.

In [38]:
# 손실 계산
preds = model(inputs)
loss = mse(preds, targets)
print(loss)

tensor(179.2473, grad_fn=<DivBackward0>)


손실은 초기에 비해 훨씬 작아졌습니다. 이제 모델의 예측을 실제 목표값과 비교해 봅시다.

In [39]:
# 예측 결과
preds

tensor([[ 56.4906,  71.3911],
        [ 68.9742, 105.4051],
        [149.8774, 120.4526],
        [ 14.4335,  42.9404],
        [ 83.0715, 124.0730]], grad_fn=<AddBackward0>)

In [40]:
# 목표 값
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])

이제 예측이 목표 변수에 상당히 근접했습니다.<br>
여기서 약간의 epoch를 더 훈련함해서 훨씬 더 좋은 결과를 얻을 수 있습니다.

---
## Pytorch에 내장된 선형 회귀

>지금까지 선형 회귀와 경사 하강법을 기본 텐서 함수들을 이용해 구현했습니다. <br>
하지만, 이러한 작업들은 딥러닝에서 흔한 패턴이기 때문에 PyTorch는 몇 줄의 코드로 쉽게 생성하고 학습할수 있는 모델들을 여러 내장 함수들과 내장 클래스들로 제공합니다.

이제 신경망을 만들 수 있는 유용한 클래스들을 포함하는 Pytorch의 `torch.nn` 패키지를 import 해 봅시다.

In [41]:
import torch.nn as nn

이전에, 저희는 입력과 목표 값을 행렬로 표현했습니다.

In [42]:
# 입력 (온도, 강우량, 습도)
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70], 
                   [74, 66, 43], 
                   [91, 87, 65], 
                   [88, 134, 59], 
                   [101, 44, 37], 
                   [68, 96, 71], 
                   [73, 66, 44], 
                   [92, 87, 64], 
                   [87, 135, 57], 
                   [103, 43, 36], 
                   [68, 97, 70]], 
                  dtype='float32')

# 목표 값 (사과, 오렌지)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119],
                    [57, 69], 
                    [80, 102], 
                    [118, 132], 
                    [21, 38], 
                    [104, 118], 
                    [57, 69], 
                    [82, 100], 
                    [118, 134], 
                    [20, 38], 
                    [102, 120]], 
                   dtype='float32')

inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

In [43]:
inputs

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.],
        [ 74.,  66.,  43.],
        [ 91.,  87.,  65.],
        [ 88., 134.,  59.],
        [101.,  44.,  37.],
        [ 68.,  96.,  71.],
        [ 73.,  66.,  44.],
        [ 92.,  87.,  64.],
        [ 87., 135.,  57.],
        [103.,  43.,  36.],
        [ 68.,  97.,  70.]])

대규모 데이터셋을 소규모 배치로 처리하는 방법을 설명하기 위해 15가지 교육 사례를 사용합니다.

---
## 데이터셋과 데이터 로더

이제 저희는 `inputs`와 `targets` 튜플에서 행 단위로 접근 가능하고 PyTorch의 다양한 데이터셋에서 작업할 수 있는 표준 API를 제공하는 `TensorDataset`을 만들 것입니다.

In [44]:
from torch.utils.data import TensorDataset

In [45]:
# 데이터셋 정의
train_ds = TensorDataset(inputs, targets)
train_ds[0:3]

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.]]),
 tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.]]))

`TensorDataset`을 사용하면 배열 인덱싱 표기법(위 코드의 `[0:3]`)을 사용해 훈련 데이터의 일부에 접근이 가능합니다.<br>
두 개의 요소를 가진 튜플을 반환하는데 첫 요소에는 선택한 행의 입력 변수가 포함되고 두 번째 요소에는 목표 값이 포함됩니다.

훈련 중에 데이터를 미리 정의된 크기의 배치로 분할할 수 있는 `데이터 로더`도 만들 예정입니다.<br>
데이터 로더는 셔플링 및 데이터의 무작위 샘플링과 같은 기타 유틸리티도 제공합니다.

In [46]:
from torch.utils.data import DataLoader

In [47]:
# 데이터 로더 정의
batch_size = 5
train_dl = DataLoader(train_ds, batch_size, shuffle=True)

이렇게 생성된 데이터 로더를 `for` 반복문에 사용할 수 있습니다.

In [48]:
for xb, yb in train_dl:
    print(xb)
    print(yb)
    break

tensor([[ 73.,  67.,  43.],
        [ 87., 134.,  58.],
        [ 91.,  87.,  65.],
        [ 92.,  87.,  64.],
        [ 88., 134.,  59.]])
tensor([[ 56.,  70.],
        [119., 133.],
        [ 80., 102.],
        [ 82., 100.],
        [118., 132.]])


각 반복에서 데이터 로더는 지정된 배치 크기의 데이터 배치를 반환합니다.<br>
`Shuffle`을 `True`로 설정시 배치 생성 전에 훈련 데이터가 섞입니다.<br>
셔플링은 최적화 알고리즘에 대한 입력을 랜덤화 시키는데 도움이 되므로 손실을 더 빨리 줄일 수 있게 도와줍니다.

---
## nn.Linear

가중치와 편향을 수동으로 초기화 하는것 대신 PyTorch에서 제공하는 클래스 `nn.Linear`를 사용해 모델을 자동으로 정의 할 수 있습니다.

In [49]:
# 모델 정의
model = nn.Linear(3, 2)
print(model.weight)
print(model.bias)

Parameter containing:
tensor([[ 0.1739,  0.3517, -0.5711],
        [-0.2976, -0.2486,  0.3945]], requires_grad=True)
Parameter containing:
tensor([ 0.0584, -0.5492], requires_grad=True)


PyTorch의 모델에는 모델에 존재하는 모든 가중치와 편향 행렬을 포함하는 리스트를 반환하는 `.parameters`라는 메서드가 존재합니다.<br>
선형 회귀 모델의 경우 가중치 행렬과 편향 행렬이 하나씩 있습니다.

In [50]:
# 파라미터
list(model.parameters())

[Parameter containing:
 tensor([[ 0.1739,  0.3517, -0.5711],
         [-0.2976, -0.2486,  0.3945]], requires_grad=True),
 Parameter containing:
 tensor([ 0.0584, -0.5492], requires_grad=True)]

이전과 같은 방법으로 모델의 예측을 생성할 수 있습니다.

In [51]:
# 예측 생성
preds = model(inputs)
preds

tensor([[ 11.7620, -21.9682],
        [ 10.2855, -24.2619],
        [ 29.1953, -36.8757],
        [ 11.7909, -26.9972],
        [  5.8465, -17.3377],
        [ 11.5842, -22.0171],
        [  9.3627, -23.6188],
        [ 28.7981, -36.7788],
        [ 11.9687, -26.9483],
        [  5.1015, -16.6456],
        [ 10.8392, -21.3251],
        [ 10.1077, -24.3109],
        [ 30.1181, -37.5188],
        [ 12.5359, -27.6893],
        [  6.0243, -17.2887]], grad_fn=<AddmmBackward>)

---
## 손실 함수

손실함수를 직접 정의하는 것 대신, 내장 함수인 `mse_loss`를 사용할 수 있습니다.

In [52]:
# Import nn.functional
import torch.nn.functional as F

`nn.functional` 패키지는 많은 유용한 손실 함수와 유틸리티들을 포함합니다.

In [53]:
# Define loss function
loss_fn = F.mse_loss

우리의 모델의 현재 예측에 대한 손실을 계산해 봅시다.

In [54]:
loss = loss_fn(model(inputs), targets)
print(loss)

tensor(10016.9580, grad_fn=<MseLossBackward>)


## Optimizer

모델의 가중치와 편향의 경사를 사용해 수동으로 수정하는 것 대신, 여기서는 `optim.SGD`를 사용할 것입니다.<br>
SGD는 "stochastic gradient descent"의 줄임말로 _stochastic_ 이라는 단어는 샘플이 무작위 배치로 선택되었음을 나타냅니다.

In [56]:
# optimizer 정의
opt = torch.optim.SGD(model.parameters(), lr=1e-5)

`model.parameters()`는 `optim.SGD`의 인자라는 점을 주의합니다.<br>
`model.parameters()`를 인자로 사용하는 것을 통해 optimizer가 어떤 행렬을 수정해야하는지 인지할 수 있습니다.<br>
또한 저희는 파라미터들을 수정시 얼마나 수정할지에 대한 값인 _learning rate_ 또한 명시해줘야 합니다.

---
## 모델 학습하기

이제 저희는 모델을 학습할 준비가 되었습니다. 저희는 경사 하강법을 다음과 같은 절차로 구현할 수 있습니다:

1. 예측 생성

2. 손실 계산

3. 가중치와 편향의 경사를 계산합니다.

4. 경사에 작은값을 곱한 값을 가중치에 더합니다.

5. 경사를 0으로 재설정합니다.

유일한 변화는 매 반복마다 훈련 데이터를 처리하는 대신 데이터를 배치로 수행한것 입니다. 이제 주어진 Epoch 수에 대해 모델을 훈련시키는 유틸리티 함수 `fit`을 정의 해 보겠습니다.

In [57]:
# 모델을 학습하기 위한 유틸리티 함수
def fit(num_epochs, model, loss_fn, opt, train_dl):
    
    # epoch의 수만큼 반복합니다.
    for epoch in range(num_epochs):
        
        # 데이터를 batch로 학습
        for xb,yb in train_dl:
            
            # 1. 예측 생성
            pred = model(xb)
            
            # 2. 손실 계산
            loss = loss_fn(pred, yb)
            
            # 3. 경사 계산
            loss.backward()
            
            # 4. 경사를 사용해 파라미터 수정
            opt.step()
            
            # 5. 경사를 0으로 재설정
            opt.zero_grad()
        
        # 현재 상태를 출력
        if (epoch+1) % 10 == 0:
            print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

위의 코드의 참고 사항:

>* 앞에서 정의한 데이터 로더를 사용해 모든 반복에 대한 데이터 배치를 가져옵니다.
>* 매개변수(가중치 & 편향)을 수동으로 업데이트 하는 대신 `opt.step`을 사용해 업데이트를 하였고, `opt.zero_grad`로 경사를 0으로 만들었다.
>* 또한 교육 진행률을 추적하기 위해 10 epoch마다 마지막 데이터 배치의 손실을 인쇄하는 로그 문장을 추가했습니다.<br>
>`loss.item`은 손실 텐서에 저장된 실제 손실 값을 반환합니다.

이제 모델을 100 epoch 동안 학습해 봅시다.

In [58]:
fit(100, model, loss_fn, opt, train_dl)

Epoch [10/100], Loss: 272.7664
Epoch [20/100], Loss: 349.9899
Epoch [30/100], Loss: 231.2212
Epoch [40/100], Loss: 127.4240
Epoch [50/100], Loss: 157.4105
Epoch [60/100], Loss: 17.4536
Epoch [70/100], Loss: 79.4062
Epoch [80/100], Loss: 59.0421
Epoch [90/100], Loss: 40.1044
Epoch [100/100], Loss: 39.8229


모델을 사용해 예측을 생성하고 목표에 가까운지 확인하겠습니다.

In [59]:
# 예측 생성
preds = model(inputs)
preds

tensor([[ 58.7257,  71.1675],
        [ 77.4212, 100.8864],
        [126.8730, 130.8898],
        [ 29.6614,  42.6539],
        [ 88.6137, 116.1300],
        [ 57.5312,  70.2291],
        [ 76.3159, 100.9788],
        [126.6868, 131.5327],
        [ 30.8558,  43.5923],
        [ 88.7029, 117.1608],
        [ 57.6205,  71.2598],
        [ 76.2267,  99.9480],
        [127.9782, 130.7974],
        [ 29.5721,  41.6231],
        [ 89.8081, 117.0685]], grad_fn=<AddmmBackward>)

In [60]:
# 목표와 비교
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 57.,  69.],
        [ 80., 102.],
        [118., 132.],
        [ 21.,  38.],
        [104., 118.],
        [ 57.,  69.],
        [ 82., 100.],
        [118., 134.],
        [ 20.,  38.],
        [102., 120.]])

사실, 예측은 목표 값과 꽤 근접했습니다. 지금까지 한 지역의 평균 기온, 강우량 및 습도를 조사하여 사과와 오렌지의 작물 수확량을 예측하는 좋은 모델을 훈련 시켰습니다. 입력 한 줄을 포함하는 배치를 전달해 새로운 지역의 농작물 수확량을 예측하는데 모델을 사용할 수 있습니다.

In [61]:
model(torch.tensor([[75, 63, 44.]]))

tensor([[54.1749, 68.6384]], grad_fn=<AddmmBackward>)

예측된 수확량은 1 헥타르당 54.3 톤 이고 오렌지는 1 헥타르당 68.3 톤입니다.

## 기계 학습 vs 클래식 프로그래밍

>이 튜토리얼의 접근 방식은 기존에 저희가 알던 프로그래밍과 많이 다릅니다.<br>
>보통, 저희는 몇가지 입력이 필요하고 몇가지 연산을 하고 결과를 반환하는 프로그램을 작성합니다.

>이 튜토리얼에선, 매개변수(추정 & 편견)을 사용하여 입력과 출력 사이의 특정 관계를 가정하는 "모델"을 정의 했습니다.<br>
우리는 정의한 모델에게 입력 및 출력을 알고있는 모델을 보여주고 loss를 최소화하는 매개변수를 찾도록 모델을 _훈련_ 합니다.<br> 
한번 훈련이 되었다면, 그 모델은 새로운 input들에 대한 결과를 계산하는데 사용될 수 있습니다.

이 프로그래밍 패러다임은 _기계 학습_ 이라고 하는데, 데이터를 사용해 입력과 출력 사이의 관계를 파악합니다.<br>
_딥 러닝_ 은 기계학습의 한 갈래로 행렬 연산, 비 선형 활성화 함수, 경사 하강법 등 모델을 만들고 학습하는 방법들을 사용합니다.<br>
테슬라 자동차의 인공지능 디렉터 Andrej Karpathy는 [Software 2.0](https://medium.com/@karpathy/software-2-0-a64152b37c35)라는 제목의 굉장히 좋은 글을 남겼습니다. 시간이 되신다면 읽어보시기 바랍니다.

아래 그림은 [Deep Learning with Python](https://www.manning.com/books/deep-learning-with-python)이라는 책에서 가져온 기계학습과 고전 프로그래밍의 차이를 보여주는 그림입니다.

![](https://i.imgur.com/oJEQe7k.png)

다음 튜토리얼들을 하면서 이 그림을 꼭 마음속에 염두해두고 하시길 바랍니다.

---
## 연습 문제와 추가 읽을거리

저희는 이번 튜토리얼에서 다음과 같은 주제들을 다뤘습니다.

>- 선형 회귀와 경사 하강법의 소개
>- PyTorch 텐서를 이용해 선형 회귀 구현하기
>- 선형 회귀 모델을 경사 하강법 알고리즘을 통해 훈련시키기
>- PyTorch에 내장된 경사 하강법과 선형 회귀를 이용해 앞에서 다룬 내용 구현해보기

선형 회귀와 경사 하강법에 더 공부해보고 싶으시다면 다음 자료들을 읽어보시기 바랍니다:

* 경사 하강법을 시각적 & 영상화하여 설명한 자료 : https://www.youtube.com/watch?v=IHZwWFHWa-w
 
* 미분과 경사 하강법에 대한 더욱 자세한 설명은 [Udacity course의 노트](https://storage.googleapis.com/supplemental_media/udacityu/315142919/Gradient%20Descent.pdf)를 보십시오.

* 선형 회귀가 어떻게 작동하는지에 대한 동영상은 [이 포스트를 보시기 바랍니다](https://hackernoon.com/visualizing-linear-regression-with-pytorch-9261f49edb09).

* 행렬 미적분, 선형 회귀, 경사 하강법에 대한 수학적 이해에 대해서는 스탠포드의 [Andrew Ng's excellent course note](https://github.com/Cleo-Stanford-CS/CS229_Notes/blob/master/lectures/cs229-notes1.pdf)를 보시기 바랍니다.

* 여러분의 스킬을 연습 및 테스트 해보려면 캐글에서 열린 대회인 [보스턴 집 값 예측 대회](https://www.kaggle.com/c/boston-housing)를 해보십시오. (캐글에서는 이런 데이터과학 대회가 개최되므로 참여해 보시는걸 추천합니다.)

지금까지, 저희는 PyTorch에서의 선형 회귀에 대해 알아보았습니다. 다음 튜토리얼에선 PyTorch에서의 이미지와 로지스틱 회귀 분석 작업에 대해 알아보겠습니다.

## Questions for Review

이 수업에서 다루는 주제에 대한 이해도를 테스트하기 위해 다음 질문에 답해 보세요.

1. 선형 회귀 모델이란 무엇입니까? 선형 회귀 모델로 공식화된 문제의 예를 들어 설명해 보세요.
2. 데이터 셋(Dataset)의 입력(input) 및 대상(target)는 무엇입니까? 예를 들어서 설명해 보세요.
3. 선형 회귀 모델에서 가중치(weight)와 편향(bias)이란 무엇입니까?
4. PyTorch 텐서를 사용하여 데이터를 어떻게 표현합니까?
5. 선형 회귀 모델(linear regression model)을 훈련하는 동안 입력(input)과 목표(targets)에 대해 별도의 행렬을 만드는 이유는 무엇인지 설명해 보세요.
6. 일부 훈련 데이터가 주어지면 가중치 행렬 및 편향 벡터의 모양을 어떻게 결정하는지 표현해 보세요.
7. 특정 모양으로 무작위로 초기화된 가중치 및 편향을 어떻게 생성하는지 표현해 보세요.
8. 행렬 연산을 사용하여 선형 회귀 모델(linear regression model)을 어떻게 구현하는지 예를 들어 설명하십시오.
10. 선형 회귀 모델을 사용하여 예측(prediction)을 어떻게 생성하는지 설명해 보세요.
12. 무작위로 초기화된 모델의 예측이 실제 목표와 다른 이유는 무엇인지 설명해 보세요.
13. 손실 함수(Loss function)란 무엇입니까? "손실(loss)"라는 의미는 무엇을 의미하는지 설명해 보세요.
14. 평균 제곱 오차(Mean Square Error)란 무엇입니까?
15. 모델 예측과 실제 목표를 사용하여 평균 제곱을 계산하는 함수를 작성하십시오.
16. 평균 제곱 오차 손실 함수의 결과에 대해 `.backward` 함수를 호출하면 어떻게 됩니까?
17. Loss의 도함수(미분)가 가중치 행렬(Matrix) 자체가 행렬인 이유를 설명하시고, 그 요소는 무엇을 나타내는지 설명하세요.
19. Loss의 도함수 Loss을 줄이는 데 어떻게 유용한 가중치 요소인지 예를 들어 설명하십시오.
20. 손실 가중치 요소의 도함수가 양수(혹은 음수)라고 가정해보고 더 낮은 손실을 얻으려면 요소의 값을 약간 늘리거나 줄여야 하는지 설명해 보세요.(음수일 경우와 양수일 경우 모두 적어보세요.)
22. 손실을 약간 줄이기 위해 각각의 기울기를 사용하여 모델의 가중치와 편향을 어떻게 업데이트합니까?
23. 경사하강법 최적화 알고리즘(gradient optimization algorithm)이란 무엇입니까? 왜 "경사하강법(gradient descent)"이라고 합니까?
24. 실제 기울기 자체가 아니라 가중치 및 편향에서 기울기에 비례하는 "소량(small quantity)"을 감소시키는 이유는 무엇입니까?
26. 학습률(Learning Rate)이란 무엇인지 쓰시고 왜 중요한지 설명해 보세요.
27. 'torch.no_grad'가 무엇인가요?
28. 가중치와 편향을 업데이트한 후 기울기를 0으로 재설정하는 이유는 무엇인가요?
25. 경사하강법을 사용하여 선형 회귀 모델을 훈련하는 단계는 무엇인가요?
26. Epoch이란 무엇입니까?
27. 여러 Epoch에 대해 모델을 훈련하면 어떤 이점이 있는지 설명해 보세요.
28. 훈련된 모델을 사용하여 어떻게 예측하는지 설명해 보세요.
29. 훈련하는 동안 모델의 손실이 줄어들지 않으면 어떻게 해야 합니까? 힌트: 학습률(Learning Rate).
30. 'torch.nn'이 무엇인지 설명해 보세요.
31. PyTorch에서 'TensorDataset' 클래스의 목적은 무엇인지 설명해 보세요. 예를 들어 주십시오.
32. PyTorch에서 Data Loader란 무엇인지 예를들어 설명해 보세요.
33. 데이터 로더를 사용하여 데이터 배치를 검색하는 방법은 무엇인지 설명해 보세요.
34. 배치를 생성하기 전에 훈련 데이터를 섞으면 어떤 이점이 있는지 설명해 보세요.
35. 전체 데이터 세트로 훈련하는 대신 소규모 배치로 훈련하면 어떤 이점이 있는지 설명해 보세요.
36. PyTorch에서 `nn.Linear` 클래스의 목적은 무엇인지 예를 들어 주십시오.
37. `nn.Linear` 모델의 가중치와 편향을 어떻게 보는지 설명해 보세요.
38. `torch.nn.functional` 모듈의 목적은 무엇인지 설명해 보세요.
39. PyTorch 내장 함수를 사용하여 평균 제곱 오차 손실을 어떻게 계산하는지 설명해 보세요.
40. PyTorch에서 Optimizer란 무엇인지 설명해 보세요.
41. 'torch.optim.SGD'가 무엇인지, SGD가 무엇을 의미하는지 설명해 보세요.
42. PyTorch 옵티마이저에 대한 입력은 무엇인지 설명해 보세요.
43. 선형 회귀 모델을 훈련하기 위한 최적화 프로그램을 만드는 예를 들어 보십시오.
44. 경사하강법을 사용하여 일괄적으로 `nn.Linear` 모델을 훈련시키는 함수를 작성하십시오.
45. 이전에 본 적이 없는 데이터를 예측하기 위해 선형 회귀 모델을 어떻게 사용합니까?